Demonstrate calibrate with inputPhase=True 

In [ ]:
import importlib
from mkids import TopSoc
import numpy as np
import matplotlib.pyplot as plt
import Scan
soc = TopSoc()

In [ ]:
fMixer = soc.fsIn/2/2
soc.set_mixer(fMixer)

In [ ]:
importlib.reload(Scan)
scan = Scan.Scan(soc)
f0 = fMixer + 85.4
f1 = fMixer + 95.6
calibration = scan.makeCalibration(fMixer, f0, f1, nf=51, nt=10, doProgress=True)



In [ ]:
iTone = 2
fTone = calibration['freqs'][iTone]
outCh = scan.soc.outFreq2ch(fTone)
print(fTone, outCh)
nominalDelay = scan.measureNominalDelay(outCh, decimation=2, doProgress=True, doPlot=True)

In [ ]:
Scan.fscanPlot(calibration, iTone)


In [ ]:

Scan.applyDelay(calibration, nominalDelay)

In [ ]:
Scan.fscanPlot(calibration, iTone)


In [ ]:
spectrum = Scan.fscanToSpectrum(calibration)

In [ ]:
plt.plot(spectrum[0],spectrum[2])
fList = scan.makeFList(fMixer, f0, f1)
for f in fList:
    plt.axvline(f,color='r')